# Mobile platform tutorial 🚗

This notebook shows how to construct a mobile platform with a UR5e arm mounted on top.

It is recommended that you follow the `01_building_a_scene.ipynb` notebook before this one.

## 1. Building the `scene` 🎭

The mobile platform that is supported in airo-drake is based on the KELO modular platform.
To create a scene with a mobile robot, you need to supply which bricks should go where. The API currently assumes that you have one or more drive bricks and at least a battery and a CPU brick.

Moreover, for collision checking, you can define the dimensions of the entire mobile platform. Boxes will be added to model the sides and roof of the robot.

At AIRO, we have a specific configuration of the KELO bricks, and the default values of the function you will see below reflect our platform.

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from pydrake.math import RigidTransform, RollPitchYaw
from airo_drake import MobilePlatformWithSingleArmScene, add_mobile_platform, finish_build, add_meshcat, add_manipulator, attach_mobile_platform_to_world

robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
mobi_index = add_mobile_platform(robot_diagram_builder, drive_positions=(np.array([1, -0.5]), np.array([1, 0.5]), np.array([-1, -0.5]),
                                                np.array([-1, 0.5])), cpu_position=np.array([0, -0.5]), battery_position=np.array([0, 0.5]))
mobi_frame = robot_diagram_builder.plant().GetFrameByName("base_link", mobi_index)

The mobile platform itself has been constructed. We still need to perform two steps:

1. We should add the UR5e arm to the robot
2. We should attach the robot to the world

In AIRO's robot, the UR5e arm should be mounted on top of the roof, at `(x, y) = (0.2445, 0)`. The arm is rotated by -90 degrees.

In [ ]:
# For these two values, see the API of `add_mobile_platform`.
side_height = 0.43
roof_thickness = 0.03
arm_transform = RigidTransform(p=[0.2445, 0, side_height + roof_thickness], rpy=RollPitchYaw([0, 0, -np.pi / 2]))

arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur5e", "robotiq_2f_85", arm_transform, static_gripper=True, parent_frame=mobi_frame)

We attach the mobile platform to the world by means of a `PlanarJoint`. This allows the robot to move with linear `x` and `y` velocities and an angular velocity `theta`. airo-drake provides a function for this.

In [ ]:
attach_mobile_platform_to_world(robot_diagram_builder, mobi_index)

In [ ]:
robot_diagram, context = finish_build(robot_diagram_builder)

scene = MobilePlatformWithSingleArmScene(robot_diagram, mobi_index, arm_index, gripper_index, meshcat)
scene

You can now look at the mobile robot in meshcat by clicking the link in the first cell's output.

## 2. Moving the mobile platform 🛞

Since we attached the mobile platform to the world by means of a planar joint, we can manipulate the pose of the robot by setting the values of this joint.
When we query the plant's positions, we see that we have 9 values. There are six values for the UR5e's joints, and 3 for the mobile platform's joint.

In [ ]:
diagram = scene.robot_diagram
plant = diagram.plant()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

len(plant.GetPositions(plant_context))

In [ ]:
plant.SetPositions(plant_context, [1, 0.5, np.pi / 2, 0, 0, 0, 0, 0, 0])
diagram.ForcedPublish(context)

Looking at meshcat, you'll see that the platform has moved 1 meter along x, 50 centimeter along y and rotated 90 degrees.
Remark how the pivot is at the center of the platform.